# 3. Model Training

This notebook demonstrates training CNN models for patch classification.

## Experiments

We run several binary classification experiments:

| # | Name | Classes | Difficulty |
|---|------|---------|------------|
| 1 | Normal vs Any Tumor | 0 vs (1,2,3) | Medium |
| 2 | Normal vs Pure Tumor | 0 vs 3 | Easy |
| 3 | Slide Context | 0 vs 1 | Hard |
| 4 | Normal vs Actual Tumor | 0 vs (2,3) | Medium |
| 5 | Normal vs Boundary | 0 vs 2 | Hard |

In [1]:
# REMOVE THIS CODE ONCE WE@RE HAPPY ALL IS WORKING WELL: sys.path.insert() will work well once this all in repo
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Navigate to project folder
%cd /content/drive/MyDrive/new_work/Projects/Camelyon16

# Verify were're in the right place
!ls
# Should show: README.md  config.py  notebooks/  requirements.txt  scripts/  src/

# Install dependencies (run in Colab)
!apt-get install -y openslide-tools
!pip install openslide-python boto3 shapely scikit-image

Mounted at /content/drive
/content/drive/MyDrive/new_work/Projects/Camelyon16
config.py  models     __pycache__  requirements.txt
LICENSE    notebooks  README.md    src
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libopenslide0
Suggested packages:
  libtiff-tools
The following NEW packages will be installed:
  libopenslide0 openslide-tools
0 upgraded, 2 newly installed, 0 to remove and 41 not upgraded.
Need to get 104 kB of archives.
After this operation, 297 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libopenslide0 amd64 3.4.1+dfsg-5build1 [89.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 openslide-tools amd64 3.4.1+dfsg-5build1 [13.8 kB]
Fetched 104 kB in 1s (137 kB/s)
Selecting previously unselected package libopenslide0.
(Reading database ... 121689 files and directories currently installed.)
Prepari

In [2]:
import sys
sys.path.insert(0, '..')

import numpy as np
import tensorflow as tf
from tensorflow import keras

from config import DEFAULT_CONFIG
from src.models import run_binary_experiment, MODEL_REGISTRY

# Set seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [3]:
# Path to pre-generated 4-class dataset
# (See generate_dataset.py script to create this)
DATASET_PATH = '/content/drive/MyDrive/new_work/Projects/pathovis_project/data/camelyon16_research_4class_400k' # previously generated dataset (~100k patches per class)

# if you don't have a dataset yet:
# from src.dataset import generate_dataset
# DATASET_PATH = generate_dataset(class_targets={0: 5000, 1: 2500, 2: 2500, 3: 5000})

## 3.1 Model Architectures

We have three model architectures available:

In [4]:
# Available models
print("Available models:")
for name, builder in MODEL_REGISTRY.items():
    model = builder()
    print(f"  {name}: {model.count_params():,} parameters")
    del model

Available models:
  simple: 65,825 parameters
  subtle: 390,593 parameters
  attention: 390,850 parameters


In [5]:
# Visualize simple model architecture
from src.models.architectures import build_simple_cnn

model = build_simple_cnn()
model.summary()

Model: "simple_cnn"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 112, 112, 16)   │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 112, 112, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_11 (Activation)      │ (None, 112, 112, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 112, 112, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 56, 56, 32)     │        12,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 56, 56, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_12 (Activation)      │ (None, 56, 56, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 56, 56, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 28, 28, 64)     │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 28, 28, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_13 (Activation)      │ (None, 28, 28, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 28, 28, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (None, 64)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 65,825 (257.13 KB)

 Trainable params: 65,601 (256.25 KB)

 Non-trainable params: 224 (896.00 B)

## 3.2 Experiment 2: Normal vs Pure Tumor (Baseline)

This is the easiest experiment - distinguishing clearly normal tissue from pure tumor.

In [ ]:
# Settings to ease RAM usage
DEFAULT_CONFIG.training.max_patches_per_chunk = 400
DEFAULT_CONFIG.training.cycle_length = 2

# Run baseline experiment
results_baseline = run_binary_experiment(
    dataset_path=DATASET_PATH,
    experiment_type=2,  # Normal vs Pure Tumor
    model_name='subtle',
    epochs=20,
    learning_rate=1e-4
)

print(f"\nBaseline Results:")
print(f"  Accuracy: {results_baseline['results']['accuracy']:.1%}")
print(f"  AUC: {results_baseline['results']['auc']:.3f}")


EXPERIMENT: Normal vs Pure Tumor
Model: subtle
Mapping: {0: ['normal_from_normal'], 1: ['pure_tumor']}
✓ No slide leakage (170 train, 78 val slides)
Train: 92 chunks, Val: 40 chunks
Steps: 1150 train, auto val

Model: 390,593 parameters
Training: 1150 steps/epoch, 20 max epochs
Epoch 1/20
1150/1150 ━━━━━━━━━━━━━━━━━━━━ 0s 523ms/step - accuracy: 0.8341 - auc: 0.9047 - loss: 0.3920
Epoch 1: val_loss improved from inf to 0.73703, saving model to ./models/normal_vs_pure_tumor.keras


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


1150/1150 ━━━━━━━━━━━━━━━━━━━━ 1054s 852ms/step - accuracy: 0.8341 - auc: 0.9047 - loss: 0.3920 - val_accuracy: 0.7075 - val_auc: 0.8280 - val_loss: 0.7370 - learning_rate: 1.0000e-04
Epoch 2/20
1150/1150 ━━━━━━━━━━━━━━━━━━━━ 0s 441ms/step - accuracy: 0.8345 - auc: 0.9079 - loss: 0.4012
Epoch 2: val_loss improved from 0.73703 to 0.56106, saving model to ./models/normal_vs_pure_tumor.keras
1150/1150 ━━━━━━━━━━━━━━━━━━━━ 840s 731ms/step - accuracy: 0.8345 - auc: 0.9079 - loss: 0.4012 - val_accuracy: 0.7124 - val_auc: 0.7816 - val_loss: 0.5611 - learning_rate: 1.0000e-04
Epoch 3/20
1150/1150 ━━━━━━━━━━━━━━━━━━━━ 0s 420ms/step - accuracy: 0.8471 - auc: 0.8922 - loss: 0.3707
Epoch 3: val_loss did not improve from 0.56106
1150/1150 ━━━━━━━━━━━━━━━━━━━━ 822s 716ms/step - accuracy: 0.8471 - auc: 0.8922 - loss: 0.3707 - val_accuracy: 0.6813 - val_auc: 0.8297 - val_loss: 0.6115 - learning_rate: 1.0000e-04
Epoch 4/20
1150/1150 ━━━━━━━━━━━━━━━━━━━━ 0s 400ms/step - accuracy: 0.8270 - auc: 0.8939 - 

In [ ]:
# Cleanup memory before next experiment
import gc
import tensorflow as tf

# Clear Keras session (releases GPU memory and graph)
tf.keras.backend.clear_session()

# Force garbage collection
gc.collect()

print("Memory cleared")

## 3.3 Experiment 3: Slide Context Detection (Hard)

Can we distinguish normal tissue from normal slides vs normal tissue from tumor slides?

This tests whether there are subtle changes in "normal" tissue when a tumor is present nearby.

In [ ]:
# Cleanup memory before next experiment
tf.keras.backend.clear_session()
gc.collect()
print("Memory cleared")

In [ ]:
# Settings to ease RAM usage
DEFAULT_CONFIG.training.max_patches_per_chunk = 400
DEFAULT_CONFIG.training.cycle_length = 2

# Run slide context experiment with subtle model
results_context = run_binary_experiment(
    dataset_path=DATASET_PATH,
    experiment_type=3,  # Slide Context
    model_name='subtle',  # Use subtle model for fine-grained features
    epochs=20,
    learning_rate=5e-5  # Lower LR for subtle task
)

print(f"\nSlide Context Results:")
print(f"  Accuracy: {results_context['results']['accuracy']:.1%}")
print(f"  AUC: {results_context['results']['auc']:.3f}")

In [ ]:
# Cleanup memory before visualization
tf.keras.backend.clear_session()
gc.collect()
print("Memory cleared")

## 3.4 Experiment 5: Normal vs Boundary (Hard)

Can we detect the transition zone between normal and tumor tissue?

In [ ]:
# Run boundary detection experiment
results_boundary = run_binary_experiment(
    dataset_path=DATASET_PATH,
    experiment_type=5,  # Normal vs Boundary
    model_name='subtle',
    epochs=20,
    learning_rate=5e-5
)

print(f"\nBoundary Detection Results:")
print(f"  Accuracy: {results_boundary['results']['accuracy']:.1%}")
print(f"  AUC: {results_boundary['results']['auc']:.3f}")

## 3.5 Compare Results

In [ ]:
import matplotlib.pyplot as plt

experiments = [
    ('Normal vs Pure Tumor', results_baseline),
    ('Slide Context', results_context),
    ('Normal vs Boundary', results_boundary)
]

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

names = [e[0] for e in experiments]
accuracies = [e[1]['results']['accuracy'] for e in experiments]
aucs = [e[1]['results']['auc'] for e in experiments]

x = np.arange(len(names))
width = 0.35

axes[0].bar(x, accuracies)
axes[0].set_ylabel('Accuracy')
axes[0].set_xticks(x)
axes[0].set_xticklabels(names, rotation=15, ha='right')
axes[0].set_ylim(0.5, 1.0)
axes[0].axhline(0.5, color='gray', linestyle='--', label='Random')

axes[1].bar(x, aucs)
axes[1].set_ylabel('AUC')
axes[1].set_xticks(x)
axes[1].set_xticklabels(names, rotation=15, ha='right')
axes[1].set_ylim(0.5, 1.0)
axes[1].axhline(0.5, color='gray', linestyle='--', label='Random')

plt.suptitle('Experiment Comparison')
plt.tight_layout()
plt.show()

## Summary

Key findings:
1. **Pure tumor detection is easy** - Clear visual differences enable high accuracy
2. **Slide context detection is hard** - Subtle changes in "normal" tissue are difficult to detect
3. **Boundary detection varies** - Performance depends on how we define "boundary"

## Next Steps

To improve hard experiments:
- **More data** - Larger datasets help with subtle patterns
- **Better augmentation** - Color normalization, stain augmentation
- **Pretrained models** - Transfer learning from ImageNet or pathology models
- **Multi-instance learning** - Consider multiple patches per slide